In [13]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
import requests
from datetime import datetime, timedelta
import os
from google.cloud import storage

# Configuração para o Google Cloud Storage
project_id = "coastal-stream-464319-q3"
gcs_bucket = "teste_ifood"
gcs_path = f"gs://{gcs_bucket}/nyc_data/"

# Inicializar cliente Storage
storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(gcs_bucket)

# Diretório temporário local
temp_dir = "/tmp/nyc_data/"

# Criar o diretório temporário
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    print(f"Diretório temporário criado: {temp_dir}")

# Função para baixar arquivos
def download_file(url, local_path):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(local_path, 'wb') as file:
            file.write(response.content)
        print(f"Arquivo baixado para {local_path}")
        return True
    except Exception as e:
        print(f"Erro ao baixar arquivo de {url}: {str(e)}")
        return False

# Função para fazer upload para o GCS
def upload_to_gcs(local_path, gcs_blob_name):
    try:
        blob = bucket.blob(gcs_blob_name)
        blob.upload_from_filename(local_path)
        print(f"Arquivo enviado para gs://{gcs_bucket}/{gcs_blob_name}")
        return f"gs://{gcs_bucket}/{gcs_blob_name}"
    except Exception as e:
        print(f"Erro ao fazer upload para GCS: {str(e)}")
        return None

# Inicializar Spark com suporte a Delta
print("Iniciando sessão Spark...")
spark = DataprocSparkSession.builder \
    .appName("NYC Taxi Data Delta") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()
print("Sessão Spark iniciada")

# Parâmetros
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
file_names = ["yellow_tripdata_", "green_tripdata_"]
start_date = "2023-01"
end_date = "2023-05"

# Gerar lista de meses
start = datetime.strptime(start_date, "%Y-%m")
end = datetime.strptime(end_date, "%Y-%m")
months = []
current = start
while current <= end:
    months.append(current.strftime("%Y-%m"))
    current += timedelta(days=32)
    current = current.replace(day=1)

print(f"Processando meses: {', '.join(months)}")

# Loop para cada combinação de arquivo e mês
for file_name in file_names:
    # Extrair o nome da pasta removendo o underscore final
    folder_name = file_name.rstrip("_")
    print(f"\nProcessando arquivos {folder_name}...")

    for month in months:
        file_url = f"{base_url}{file_name}{month}.parquet"
        local_file_path = f"{temp_dir}{file_name}{month}.parquet"
        gcs_temp_path = f"nyc_data/temp/{file_name}{month}.parquet"
        delta_path = f"{gcs_path}{folder_name}_delta/{month}"

        print(f"Baixando {file_name}{month}...")
        if download_file(file_url, local_file_path):
            # Fazer upload para o GCS temporariamente
            print(f"Enviando {file_name}{month} para o GCS...")
            gcs_uri = upload_to_gcs(local_file_path, gcs_temp_path)

            if gcs_uri:
                print(f"Lendo {file_name}{month} com Spark do GCS...")
                try:
                    # Ler o arquivo Parquet do GCS com Spark
                    df = spark.read.parquet(gcs_uri)

                    # Mostrar quantidade de registros e schema
                    count = df.count()
                    print(f"Arquivo {file_name}{month} contém {count} registros")
                    print("Estrutura do DataFrame:")
                    df.printSchema()

                    # Salvar como Delta Lake
                    print(f"Salvando {file_name}{month} como Delta Lake em {delta_path}...")
                    df.write \
                        .format("delta") \
                        .mode("overwrite") \
                        .save(delta_path)
                    print(f"✓ Arquivo {file_name}{month} salvo como Delta Lake em {delta_path}")

                except Exception as e:
                    print(f"✗ Erro ao processar {file_name}{month} com Spark: {str(e)}")
            else:
                print(f"✗ Falha no upload para GCS, pulando processamento")
        else:
            print(f"✗ Falha ao baixar {file_name}{month}, pulando processamento")

        # Remover arquivo local para economizar espaço
        if os.path.exists(local_file_path):
            os.remove(local_file_path)
            print(f"Arquivo local {local_file_path} removido")

# Encerrar a sessão Spark
spark.stop()
print("\nProcessamento concluído!")

Iniciando sessão Spark...
Using existing Dataproc Session (configuration changes may not be applied): https://console.cloud.google.com/dataproc/interactive/southamerica-east1/sc-20250811-210502-8nv3ck?project=coastal-stream-464319-q3
Sessão Spark iniciada
Processando meses: 2023-01, 2023-02, 2023-03, 2023-04, 2023-05

Processando arquivos yellow_tripdata...
Baixando yellow_tripdata_2023-01...
Arquivo baixado para /tmp/nyc_data/yellow_tripdata_2023-01.parquet
Enviando yellow_tripdata_2023-01 para o GCS...
Arquivo enviado para gs://teste_ifood/nyc_data/temp/yellow_tripdata_2023-01.parquet
Lendo yellow_tripdata_2023-01 com Spark do GCS...


Arquivo yellow_tripdata_2023-01 contém 3066766 registros
Estrutura do DataFrame:
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

Salvando yellow_tripdata_2023-01 como Delta Lake 

✓ Arquivo yellow_tripdata_2023-01 salvo como Delta Lake em gs://teste_ifood/nyc_data/yellow_tripdata_delta/2023-01
Arquivo local /tmp/nyc_data/yellow_tripdata_2023-01.parquet removido
Baixando yellow_tripdata_2023-02...
Arquivo baixado para /tmp/nyc_data/yellow_tripdata_2023-02.parquet
Enviando yellow_tripdata_2023-02 para o GCS...
Arquivo enviado para gs://teste_ifood/nyc_data/temp/yellow_tripdata_2023-02.parquet
Lendo yellow_tripdata_2023-02 com Spark do GCS...


Arquivo yellow_tripdata_2023-02 contém 2913955 registros
Estrutura do DataFrame:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

Salvando yellow_tripdata_2023-02 como Delta 

✓ Arquivo yellow_tripdata_2023-02 salvo como Delta Lake em gs://teste_ifood/nyc_data/yellow_tripdata_delta/2023-02
Arquivo local /tmp/nyc_data/yellow_tripdata_2023-02.parquet removido
Baixando yellow_tripdata_2023-03...
Arquivo baixado para /tmp/nyc_data/yellow_tripdata_2023-03.parquet
Enviando yellow_tripdata_2023-03 para o GCS...
Arquivo enviado para gs://teste_ifood/nyc_data/temp/yellow_tripdata_2023-03.parquet
Lendo yellow_tripdata_2023-03 com Spark do GCS...


...

✓ Arquivo green_tripdata_2023-03 salvo como Delta Lake em gs://teste_ifood/nyc_data/green_tripdata_delta/2023-03
Arquivo local /tmp/nyc_data/green_tripdata_2023-03.parquet removido
Baixando green_tripdata_2023-04...
Arquivo baixado para /tmp/nyc_data/green_tripdata_2023-04.parquet
Enviando green_tripdata_2023-04 para o GCS...
Arquivo enviado para gs://teste_ifood/nyc_data/temp/green_tripdata_2023-04.parquet
Lendo green_tripdata_2023-04 com Spark do GCS...


Arquivo green_tripdata_2023-04 contém 65392 registros
Estrutura do DataFrame:
root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

Salvando g

✓ Arquivo green_tripdata_2023-04 salvo como Delta Lake em gs://teste_ifood/nyc_data/green_tripdata_delta/2023-04
Arquivo local /tmp/nyc_data/green_tripdata_2023-04.parquet removido
Baixando green_tripdata_2023-05...
Arquivo baixado para /tmp/nyc_data/green_tripdata_2023-05.parquet
Enviando green_tripdata_2023-05 para o GCS...
Arquivo enviado para gs://teste_ifood/nyc_data/temp/green_tripdata_2023-05.parquet
Lendo green_tripdata_2023-05 com Spark do GCS...


Arquivo green_tripdata_2023-05 contém 69174 registros
Estrutura do DataFrame:
root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

Salvando g

✓ Arquivo green_tripdata_2023-05 salvo como Delta Lake em gs://teste_ifood/nyc_data/green_tripdata_delta/2023-05
Arquivo local /tmp/nyc_data/green_tripdata_2023-05.parquet removido

Processamento concluído!
